In [46]:
import music21
import collections
import numpy as np
from os import listdir

In [2]:
music21.environment.Environment('converter.subConverters')['musescoreDirectPNGPath'] = 'C:/Program Files/MuseScore 3/bin/MuseScore3.exe'

In [5]:
bach_path = 'D:/pet_prj/nn_env/Lib/site-packages/music21/corpus/bach/'
bach_list = listdir(bach_path)
music21.corpus.addPath(bach_path)

In [8]:
smp = music21.corpus.parse(bach_list[0])

In [22]:
horn = smp.getElementsByClass('Part')[0]

In [30]:
horn_notes = horn.flatten().getElementsByClass('Note')

In [123]:
horn_key = horn.flatten().getElementsByClass('Key')[0]

In [131]:
root = horn_key.chord[0]

In [182]:
music21.interval.Interval(horn_key.chord[1], horn_key.chord[0]).name

'M2'

In [187]:
set_of_intervals = []
for n, step in enumerate(horn_key.chord):
    if n>0:
        print(n, step)
        set_of_intervals.append(music21.interval.Interval(step, horn_key.chord[n-1]).name)

1 <music21.note.Note G>
2 <music21.note.Note A>
3 <music21.note.Note B->
4 <music21.note.Note C>
5 <music21.note.Note D>
6 <music21.note.Note E>
7 <music21.note.Note F>


In [188]:
set_of_intervals

['M2', 'M2', 'm2', 'M2', 'M2', 'M2', 'm2']

In [190]:
net = music21.scale.intervalNetwork.IntervalNetwork()

In [191]:
net.fillBiDirectedEdges(set_of_intervals)

In [331]:
net

In [202]:
root.name

'F'

In [270]:
def getAllScale(scale):
    root = scale[0]
    set_of_intervals = []
    for n, step in enumerate(scale):
        if n>0:
            set_of_intervals.append(music21.interval.Interval(step, scale[n-1]).name)
    net = music21.scale.intervalNetwork.IntervalNetwork()
    net.fillBiDirectedEdges(set_of_intervals)
    octaves = [x for x in range(9)]
    notes = []
    for octave in octaves:
        current_root = root.name + str(octave)
        for p in net.realizePitch(current_root):
            if str(p) not in notes:
                notes.append(str(p))
    return np.array(notes)

In [221]:
hornScale = getAllScale(horn_key.chord)

In [33]:
note_0 = horn_notes[0]

In [136]:
interval = music21.interval.Interval(horn_notes[2], root)

In [164]:
interval.niceName

'Perfect Fourth'

In [137]:
interval.name

'P4'

In [34]:
note_0.nameWithOctave

'F4'

In [64]:
note_0.nameWithOctave + str(note_0.beatDuration)

'F4<music21.duration.Duration 1.0>'

In [78]:
fnote = horn_notes[5].fullName

In [108]:
note_0.pitch.midi

65

In [117]:
horn_notes[2].pitch.pitchClassString

'0'

In [226]:
np.where(hornScale==note_0.nameWithOctave)[0][0]

28

In [239]:
note_0.pitch.midi

65

In [240]:
music21.note.Note(65)

<music21.note.Note F>

In [269]:
def scalePosition(note, scale):
    if note.nameWithOctave in scale:
        return np.where(scale==note.nameWithOctave)[0][0]
    note_midi = note.pitch.midi
    demNote = music21.note.Note(note_midi - 1)
    if demNote.nameWithOctave in scale:
        return np.where(scale==demNote.nameWithOctave)[0][0] + 0.5
    accNote = music21.note.Note(note_midi + 1)
    if accNote.nameWithOctave in scale:
        return np.where(scale==accNote.nameWithOctave)[0][0] - 0.5

In [268]:
def noteToDict(note, root, scale):
    return {
        'name':note.name,
        'octave':note.octave,
        'length':note.fullName.split()[-2],
        'midi':note.pitch.midi,
        'interval':music21.interval.Interval(note, root).name,
        'intervalSF':music21.interval.Interval(note, root).directedNiceName,
        'scalePosition':scalePosition(note, scale)
    }

In [275]:
def motiveToDict(notes):
    def getMotive(arr):
        result=[]
        for n, i in enumerate(arr[1:]):
            result.append(i - arr[n])
        return result
    
    return {
        'notes':[x['name'] for x in notes],
        'octaves':[x['octave'] for x in notes],
        'rythm':[x['length'] for x in notes],
        'scalePositions':getMotive([x['scalePosition'] for x in notes]),
        'midi':getMotive([x['midi'] for x in notes])
    }

In [279]:
def getMotives(motiveLen, notes):
    iter_num = len(notes) - motiveLen + 1
    result = []
    for i in range(iter_num):
        result.append(motiveToDict(notes[i:i+motiveLen]))
    return result

In [327]:
def hashMotive(m):
    result = ''
    for k in m:
        result+='|'
        result+=':'.join([str(x) for x in m[k]])
    return result[1:]

In [329]:
def unHashMotive(m):
    arr = m.split('|')
    return {
        'notes':arr[0].split(':'),
        'octaves':[int(x) for x in arr[1].split(':')],
        'rythm':arr[2].split(':'),
        'scalePositions':[float(x) for x in arr[3].split(':')],
        'midi':[int(x) for x in arr[4].split(':')]        
    }

In [284]:
def isFullEqual(m1, m2):
    if m1['notes']==m2['notes'] and m1['octaves']==m2['octaves'] and m1['rythm']==m2['rythm']:
        return True
    return False

In [310]:
def grafMotives(motives):
    result = []
    for n, m1 in enumerate(motives[1:]):
        sub_motives=motives[n+1:]
        for m2 in sub_motives:
            if isFullEqual(m1, m2):
                if m1 not in result:
                    result.append(m1)
    return result

In [271]:
notesInDict = []
for note in horn_notes:
    #print(note.nameWithOctave)
    notesInDict.append(noteToDict(note, root, hornScale))

In [305]:
hornMotive_3len = getMotives(3, notesInDict)
print(len(hornMotive_3len))

151


In [311]:
feq_ms = grafMotives(hornMotive_3len)
print(len(feq_ms))

89


In [315]:
feq_ms[0]

{'notes': ['G', 'C', 'F'],
 'octaves': [4, 4, 4],
 'rythm': ['Eighth', 'Eighth', 'Eighth'],
 'scalePositions': [-4, 3],
 'midi': [-7, 5]}

In [328]:
hm = hashMotive(feq_ms[0])
print(hm)

G:C:F|4:4:4|Eighth:Eighth:Eighth|-4:3|-7:5


In [330]:
unHashMotive(hm)

{'notes': ['G', 'C', 'F'],
 'octaves': [4, 4, 4],
 'rythm': ['Eighth', 'Eighth', 'Eighth'],
 'scalePositions': [-4.0, 3.0],
 'midi': [-7, 5]}

In [290]:
m1 = hornMotive_3len[44]
m2 = hornMotive_3len[1]
isFullEqual(m1, m2)

False

In [298]:
hornMotive_3len[48]

{'notes': ['F', 'G', 'C'],
 'octaves': [4, 4, 4],
 'rythm': ['Quarter', 'Eighth', 'Eighth'],
 'scalePositions': [1, -4],
 'midi': [2, -7]}

In [276]:
motiveToDict(notesInDict[:3])

{'notes': ['F', 'G', 'C'],
 'octaves': [4, 4, 4],
 'rythm': ['Quarter', 'Eighth', 'Eighth'],
 'scalePositions': [1, -4],
 'midi': [2, -7]}

In [264]:
'|'.join([x['name'] for x in notesInDict[:3]])

'F|G|C'

In [265]:
[x['name'] for x in notesInDict[:3]]

['F', 'G', 'C']

In [102]:
horn_notes[2].fullName.split()[-2]

'Eighth'

In [81]:
notes_with_octave = []
for n in horn_notes:
    notes_with_octave.append(n.fullName)

In [70]:
def calcSeq(arrSeq, lenSeq):
    iter_num = len(arrSeq) - lenSeq + 1
    seqDict = {}
    for i in range(iter_num):
        seqName = '|'.join(arrSeq[i:i+lenSeq])
        if seqName not in seqDict:
            seqDict[seqName] = 0
        seqDict[seqName]+=1
    return {k: v for k, v in sorted(seqDict.items(), key=lambda item: item[1], reverse=True)}

In [40]:
a = ['a', 'v', 'c']

In [43]:
'|'.join(a)

'a|v|c'

In [380]:
class Xnote:
    """
    """
    def __init__(self, note, scale_intervals, root, context=None):    
        self.step = self.getStep(note, self.getFullScale(scale_intervals, root))
        self.length = note.fullName.split()[-2]
        
        
    def getFullScale(self, scale_intervals, root):
        net = music21.scale.intervalNetwork.IntervalNetwork()
        net.fillBiDirectedEdges(scale_intervals)
        octaves = [x for x in range(9)]
        notes = []
        for octave in octaves:
            current_root = root.name + str(octave)
            for p in net.realizePitch(current_root):
                if str(p) not in notes:
                    notes.append(str(p))
        return np.array(notes)


    def getStep(self, note, fullScale):
        if note.nameWithOctave in fullScale:
            return np.where(fullScale==note.nameWithOctave)[0][0]
        note_midi = note.pitch.midi
        demNote = music21.note.Note(note_midi - 1)
        if demNote.nameWithOctave in fullScale:
            return np.where(fullScale==demNote.nameWithOctave)[0][0] + 0.5
        accNote = music21.note.Note(note_midi + 1)
        if accNote.nameWithOctave in fullScale:
            return np.where(fullScale==accNote.nameWithOctave)[0][0] - 0.5
        
    def getNote(self, scale_intervals, root):
        noteName = self.getFullScale(scale_intervals, root)[self.step]
        note = music21.note.Note(noteName)
        note.duration = music21.duration.Duration(self.length.lower())
        return note

In [407]:
class Xmotive:
    """
    """
    def __init__(self, xnotes):
        self.notes = xnotes
        
        
    def getNotes(self, scale_intervals, root):
        return [note.getNote(scale_intervals, root) for note in self.notes]
    
    
    def getSteps(self):
        return [note.step for note in self.notes]

    def getRhythm(self):
        return [note.length for note in self.notes]
    
    def getMelodyLine(self):
        line = []
        steps = self.getSteps()
        for n, step in enumerate(steps[1:]):
            line.append(step - steps[n])
        return line

In [388]:
xn = Xnote(note_2, set_of_intervals, root)

In [389]:
note_x = xn.getNote(set_of_intervals, root)

In [391]:
note_x

<music21.note.Note C>

In [387]:
note_2 = horn_notes[2]

In [379]:
note_x.quarterLength

2.0

In [370]:
note_0.quarterLength

1.0

In [408]:
xm = Xmotive([Xnote(note, set_of_intervals, root) for note in horn_notes[:3]])

In [409]:
xm.getSteps()

[28, 29, 25]

In [410]:
xm.getMelodyLine()

[1, -4]

In [406]:
xm.getRhythm()

['Quarter', 'Eighth', 'Eighth']

In [396]:
xm.getNotes(set_of_intervals, root)

[<music21.note.Note F>, <music21.note.Note G>, <music21.note.Note C>]

In [ ]:
music21.note.Note()